In [2]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
#API key
api_key = "RGAPI-f5235b3e-de20-4eef-b79e-f93714ce10cb"
gameName = "Slylio" #Pseudo du mec qu'on veut check
tagLine = "EUW" #son tagline

In [16]:
#player unviveraly unique identifier
requestPUUID = requests.get(f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}")
print(requestPUUID.json())
puuid = requestPUUID #TODO, actuellement la request puuid est une sorte de json, il faut en tirer uniquement le puuid

{'status': {'message': 'Forbidden', 'status_code': 403}}


In [10]:
#match history from PUUID 
nb_of_games = 20
games = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nb_of_games}&api_key={api_key}")
print(games.json())

{'status': {'message': 'Forbidden', 'status_code': 403}}
